In [1]:
from pynq import Overlay
from pynq import MMIO
import struct
import numpy as np

ol = Overlay("./design_1_wrapper.bit")
ol.download()

cordiccart2pol_ip = MMIO(0x40000000, 0x10000)


# Values for Fixed-Point scaling
total_bits = 32
integer_bits = 3
fractional_bits = total_bits - integer_bits
scaling_factor = 2**fractional_bits
sign_bit = 1 << 31

In [2]:
input_x = -1
input_y = 0

In [4]:
input_x = 0
input_y = -1

In [6]:
input_x = -0.4854
input_y = 0.7003

In [7]:
# Round and truncate to 32 bit integer
scaled_x = np.int32(np.round(input_x * scaling_factor))
scaled_y = np.int32(np.round(input_y * scaling_factor))

# Pack to a little endian 32 bit integer
packed_x = struct.pack('<i', scaled_x)
packed_y = struct.pack('<i', scaled_y)

cordiccart2pol_ip.write(0x10,packed_x)
read_x = cordiccart2pol_ip.read(0x10)

# Handle 2's complement
# https://stackoverflow.com/questions/32030412/twos-complement-sign-extension-python
read_x = (read_x & (sign_bit - 1)) - (read_x & sign_bit)

# Revert scaling for reads
print("Input X: ", read_x / scaling_factor)

cordiccart2pol_ip.write(0x18,packed_y)
read_y = cordiccart2pol_ip.read(0x18)

read_y = (read_y & (sign_bit - 1)) - (read_y & sign_bit)
    
print("Input Y: ", read_y / scaling_factor)

cordiccart2pol_ip.write(0x00, 1)

read_r = cordiccart2pol_ip.read(0x20)

read_r = (read_r & (sign_bit - 1)) - (read_r & sign_bit)


print("Output r: ", read_r / scaling_factor)

read_theta = cordiccart2pol_ip.read(0x30)

read_theta = (read_theta & (sign_bit - 1)) - (read_theta & sign_bit)

print("Output theta: ", read_theta / scaling_factor)

Input X:  -0.48540000058710575
Input Y:  0.7003000006079674
Output r:  0.9075107704848051
Output theta:  2.1768973004072905
